# Scrapping data (old code)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen as ureq
import re

In [2]:
df = pd.DataFrame(columns =['price', 'locali', 'superficie', 'bagni','piano'])

In [5]:
links=[]
for i in range(1,10):
    content = requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i))
    soup = BeautifulSoup(content.text, 'html.parser')
    for link in soup.find_all('a', href=True):
        url = link['href']
        if url.startswith('https://www.immobiliare.it/') and url.endswith('.html'):
            links.append(url)

In [63]:
content = requests.get(links[0])
soup = BeautifulSoup(content.text, 'html.parser')
features = soup.find_all(class_=["list-inline list-piped features__list"])[0].get_text()
features = features.replace('\xa0', "")
tmp = features.split("locali")
locali = int(tmp[0])
superficie = int(tmp[1].split('m2superficie')[0]) 
bagni, piano = map(int, tmp[1].split('m2superficie')[1].split()[0].split("bagni"))

# Mongo_DB

In [79]:
from pymongo import MongoClient
import json

client = MongoClient()
my_key = "AnqLqo5TT9JDfzfqKrDUEYUqJ6HWZJy8"
base = "https://api.mlab.com/api/1"
url_db = base + "/databases?apiKey=" + my_key
response = requests.get(url_db)
databases = json.loads(response.text)
adm_database = databases[0]
adm_database

'adm_hm4'

In [80]:
# structure
headers = {'content-type':'application/json'}

# test data
flat_data = [{'price':100,'locali': 2, 'superficie': 1, "bagni": 2, "piano": 5}]

In [81]:
# I manually created the collection 'flats' in "adm_hm4" database
collection = 'flats'
url = base + '/databases/' + adm_database + "/collections/" + collection +'/?apiKey=' + my_key

# Pushing data into database
payload = json.dumps(flat_data)
response = requests.post(url, data=payload, headers=headers)

In [82]:
# Checking what is inside our database
response = requests.get(url)
result = json.loads(response.text)
result

[{'_id': {'$oid': '5bffffcb1f6e4f34a8ba7023'},
  'bagni': 2,
  'locali': 2,
  'piano': 5,
  'price': 100,
  'superficie': 1}]

Now, we have to extract data from websites and push into database

In [ ]:
#extractInfo is function to scrap html ; output --> dictionary
import logging

def threadExtract(url):
    logging.info('extracting' + url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html5lib")
    work = extractInfo(soup) #output should be a dictionary
    
    # here: putting into csv or mongo_db
    # MONGO_DB structure--> [{'price':100,'locali': 2, 'superficie': 1, "bagni": 2, "piano": 5}]
    
    logging.info("Done extracting")
    
for url in links:
    threadExtract(url)